# 4.10 Task Part 2: Customer Profile and Visualizations 

## Table of Contents

01. Import Libraries

02. Import Data and Adjust Datatypes

03. Create Additional Columns for Profiling

04. Create a DF for Unique Customer ID Profiling

05. Export Visualizations and DFs

## 01. Import Libraries

In [2]:
#import libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

## 02. Import Data and Adjust Datatypes

In [3]:
#create path
path = r'/Users/juliewhitton/Desktop/Instacart Basket Analysis/'

In [4]:
#import latest dataset
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'full_set_with_flags.pkl'))

In [5]:
#check import 
df.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,family_status,income,region,activity_flag,age_category,income_category,department,pet_owner,has_babies,has_kids
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes


In [6]:
df.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'time_of_order', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_spent', 'spending_flag', 'median_days_since_last_order',
       'freq_flag', 'first_name', 'last_name', 'gender', 'state', 'age',
       'date_joined', 'number_dependents', 'family_status', 'income', 'region',
       'activity_flag', 'age_category', 'income_category', 'department',
       'pet_owner', 'has_babies', 'has_kids'],
      dtype='object')

In [7]:
#remove unnecessary columns

df.drop(['date_joined', 'reordered', 'add_to_cart_order', 'busiest_day',
         'busiest_days', 'department_id', 'aisle_id', 'product_name'], axis = 1, inplace = True)

In [8]:
#check drop

df.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,prices,price_range_loc,busiest_period_of_day,...,family_status,income,region,activity_flag,age_category,income_category,department,pet_owner,has_babies,has_kids
0,2539329,1,1,2,8,NaN,196,9.0,Mid-range product,Average orders,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
1,2398795,1,2,3,7,15.0,196,9.0,Mid-range product,Average orders,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
2,473747,1,3,3,12,21.0,196,9.0,Mid-range product,Most orders,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
3,2254736,1,4,4,7,29.0,196,9.0,Mid-range product,Average orders,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes
4,431534,1,5,4,15,28.0,196,9.0,Mid-range product,Most orders,...,married,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes


## 03. Create Additional Columns for Profiling

In [9]:
#create column for total cost of each order
df['order_cost'] = df.groupby('order_id')['prices'].transform('sum')


In [10]:
#create column for total spent/customer
df['total_spent'] = df.groupby('user_id')['order_cost'].transform('sum')

In [11]:
#check new column
df.loc[:, ['total_spent', 'order_cost', 'prices']]

,total_spent,order_cost,prices
0,2392.00,31.593750,9.000000
1,2392.00,35.406250,9.000000
2,2392.00,28.203125,9.000000
3,2392.00,26.000000,9.000000
4,2392.00,57.000000,9.000000
...,...,...,...
32404850,278.25,10.703125,10.703125
32404851,278.25,10.703125,10.703125
32404852,278.25,10.703125,10.703125
32404853,278.25,10.703125,10.703125


In [12]:
#create column for average spent per order per customer
df['avg_spent_per_order'] = df['total_spent'].div(df['max_order'].values)


In [13]:
#create order spending flag column
df.loc[df['avg_spent_per_order'] < 50, 'spending_flag_ords'] = 'Low-spender'
df.loc[(df['avg_spent_per_order'] >= 50) & (df['avg_spent_per_order'] < 200), 'spending_flag_ords'] = 'Regular-spender'
df.loc[df['avg_spent_per_order'] >= 200, 'spending_flag_ords'] = 'High-spender'

In [14]:
#check value counts of new column
df['spending_flag_ords'].value_counts(dropna = False)

High-spender       29629544
Regular-spender     1168196
Low-spender          166824
Name: spending_flag_ords, dtype: int64

## 04. Create a DF for Unique Customer ID Profiling

In [15]:
#import customer subset df

df_cust = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'cust_unique.pkl'))

In [16]:
#order spending_flag_ords col

df_ords_flag = df[['user_id', 'spending_flag_ords']].sort_values(by = ['spending_flag_ords'],
                                                                ascending = False)

In [17]:
df_ords_flag = df_ords_flag.drop_duplicates(subset = ['user_id'])

df_ords_flag['spending_flag_ords'].value_counts(dropna = False)

High-spender       137124
Regular-spender     19154
Low-spender          6353
Name: spending_flag_ords, dtype: int64

In [18]:
#order pet_owner column 
df_pets = df[['user_id', 'pet_owner']].sort_values(by=['pet_owner'], ascending=False)

df_pets.head()

,user_id,pet_owner
903482,158382,Yes
7834301,80527,Yes
14704771,182211,Yes
14704772,182211,Yes
14704773,182211,Yes


In [19]:
#update df_pets by user_id
df_pets = df_pets.drop_duplicates(subset = ['user_id'])

df_pets['pet_owner'].value_counts(dropna = False)

No     149456
Yes     13175
Name: pet_owner, dtype: int64

In [20]:
#repeat process for has_babies
df_babies = df[['user_id', 'has_babies']].sort_values(by=['has_babies'], ascending=False)

df_babies.head()

,user_id,has_babies
13095832,158226,Yes
17231297,150299,Yes
17231309,150299,Yes
22925058,101236,Yes
17231305,150299,Yes


In [21]:
#create df of unique users
df_babies = df_babies.drop_duplicates(subset = ['user_id'])

#check output
df_babies['has_babies'].value_counts(dropna = False)

No     139978
Yes     22653
Name: has_babies, dtype: int64

In [22]:
#repeat process for has_kids
df_kids = df[['user_id', 'has_kids']].sort_values(by=['has_kids'], ascending=False)

df_kids.head()

,user_id,has_kids
0,1,Yes
19577735,13915,Yes
19577745,13915,Yes
19577744,13915,Yes
19577743,13915,Yes


In [23]:
#create kids df by unique user
df_kids = df_kids.drop_duplicates(subset = ['user_id'])

#check output
df_kids['has_kids'].value_counts(dropna = False)

Yes    121904
No      40727
Name: has_kids, dtype: int64

In [24]:
#make main df for customers
df_cust = df_cust.merge(df_pets,how ='left').merge(df_babies,how ='left').merge(df_kids, how = 'left').merge(df_ords_flag,
                                                                                                            how = 'left')

df_cust.value_counts(['pet_owner', 'has_babies', 'has_kids', 'spending_flag_ords'])

pet_owner  has_babies  has_kids  spending_flag_ords
No         No          Yes       High-spender          73692
                       No        High-spender          31206
           Yes         Yes       High-spender          19655
           No          Yes       Regular-spender       13287
Yes        No          Yes       High-spender           7263
No         No          Yes       Low-spender            4614
                       No        Regular-spender        4613
Yes        No          No        High-spender           3116
           Yes         Yes       High-spender           2192
No         No          No        Low-spender            1640
           Yes         Yes       Regular-spender         689
Yes        No          Yes       Regular-spender         364
                       No        Regular-spender         145
No         Yes         Yes       Low-spender              60
Yes        Yes         Yes       Regular-spender          56
           No          Yes       

In [25]:
#check shape
df_cust.shape

(162631, 41)

In [26]:
df_cust.head()

,order_id,user_id,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,income,region,activity_flag,age_category,income_category,department,pet_owner,has_babies,has_kids,spending_flag_ords
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,40423,South,High-activity Customer,Young Adult,Low income,beverages,No,No,Yes,High-spender
1,2968173,15,15,1,9,7.0,196,2,0,Soda,...,54313,Midwest,High-activity Customer,Senior,Middle income,beverages,No,No,No,Low-spender
2,2293453,19,2,5,14,6.0,196,3,0,Soda,...,151988,South,High-activity Customer,Adult,Middle income,beverages,No,No,Yes,High-spender
3,1573906,21,10,3,10,6.0,196,2,0,Soda,...,124643,West,High-activity Customer,Senior,Middle income,beverages,No,No,No,High-spender
4,1593000,31,10,3,8,7.0,196,17,0,Soda,...,52311,South,High-activity Customer,Adult,Middle income,beverages,No,No,Yes,High-spender


In [27]:
df_cust.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'time_of_order', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'avg_spent', 'spending_flag', 'median_days_since_last_order',
       'freq_flag', 'first_name', 'last_name', 'gender', 'state', 'age',
       'date_joined', 'number_dependents', 'family_status', 'income', 'region',
       'activity_flag', 'age_category', 'income_category', 'department',
       'pet_owner', 'has_babies', 'has_kids', 'spending_flag_ords'],
      dtype='object')

In [28]:
#drop unnecessary columns

df_cust.drop(columns = ['reordered', 'product_name', 'add_to_cart_order', 'busiest_day', 'aisle_id', 'department_id',
             'busiest_days', 'busiest_period_of_day', 'first_name', 'last_name', 'date_joined'], index = 1,
            inplace = True)

In [29]:
df_cust.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'time_of_order', 'days_since_prior_order', 'product_id', 'prices',
       'price_range_loc', 'max_order', 'loyalty_flag', 'avg_spent',
       'spending_flag', 'median_days_since_last_order', 'freq_flag', 'gender',
       'state', 'age', 'number_dependents', 'family_status', 'income',
       'region', 'activity_flag', 'age_category', 'income_category',
       'department', 'pet_owner', 'has_babies', 'has_kids',
       'spending_flag_ords'],
      dtype='object')

In [30]:
#export customers to avoid flags
df_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customer_subset.pkl'))

In [31]:
#reimport to avoid flags
df_cust = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customer_subset.pkl'))

In [32]:
#check import
df_cust.shape

(162630, 30)

In [33]:
#create new column for pets and babies

conditions = [
    (df_cust['pet_owner'] == 'Yes') & (df_cust['has_babies'] == 'Yes'),
    (df_cust['pet_owner'] == 'Yes') & (df_cust['has_babies'] == 'No'),
    (df_cust['pet_owner'] == 'No') & (df_cust['has_babies'] == 'Yes'),
    (df_cust['pet_owner'] == 'No') & (df_cust['has_babies'] == 'No'),
]


results = ['Has Both', 'Pets Only', 'Babies Only', 'Neither']


df_cust['pets_babies'] = np.select(conditions, results)

In [34]:
#check new column

df_cust['pets_babies'].value_counts(dropna = False)

Neither        129051
Babies Only     20404
Pets Only       10926
Has Both         2249
Name: pets_babies, dtype: int64

In [35]:
#create new column for kids and pets

conditions = [
    (df_cust['pet_owner'] == 'Yes') & (df_cust['has_kids'] == 'Yes'),
    (df_cust['pet_owner'] == 'Yes') & (df_cust['has_kids'] == 'No'),
    (df_cust['pet_owner'] == 'No') & (df_cust['has_kids'] == 'Yes'),
    (df_cust['pet_owner'] == 'No') & (df_cust['has_kids'] == 'No'),
]

results = ['Has Both', 'Pets Only', 'Kids Only', 'Neither']
df_cust['pets_kids'] = np.select(conditions, results)

#check new column
df_cust['pets_kids'].value_counts(dropna = False)

Kids Only    111997
Neither       37458
Has Both       9907
Pets Only      3268
Name: pets_kids, dtype: int64

In [36]:
df_cust['spending_flag_ords'].value_counts()

High-spender       137124
Regular-spender     19154
Low-spender          6352
Name: spending_flag_ords, dtype: int64

In [37]:
df_cust.shape

(162630, 32)

### 05. Export Customer DF

In [38]:
#export df of individual customer set
df_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'unique_customers_final.pkl'))


In [39]:
#export full df

df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'full_set_with_flags.pkl'))
